<a href="https://colab.research.google.com/github/nakampany/DeepLearning2022/blob/main/08_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 簡単な画像分類モデルの実装
## 概要
- 深層学習ライブラリPytorchを用いることで，これまで学んだモデルを容易に構築することができる．
- 画像分類問題を対象に，モデル構築の実践練習を行う．
- データセットは0-9の手書き数字認識データセットのMNISTを用いる．

## MNIST
- 0-9の手書き数字認識データセット
- 訓練用6万枚，検証用1万枚の画像＋ラベル（0-9）のペアで構成される．
- 一枚の画像はグレースケール(1ch)の28x28のため，$\boldsymbol{x} \in R^{60000 \times 1 \times 28 \times 28}$
- http://yann.lecun.com/exdb/mnist/

In [ ]:
# MNISTデータセットをロードする。
import torch
import torchvision
from torchvision import datasets, transforms
'''  # 大学内のProxy環境の場合，以下のコードを実行する必要があるかも．
datasets.MNIST.resources = [
    ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
    ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
]
'''

# データ拡張として標準化とTensor化を行う．
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5, ), (0.5, ))])
ds_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
ds_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader を作成する。
batch_size = 128  # バッチサイズ
#batch_size = 512  # メモリが小さい環境の場合，この値を小さくしてね．
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_test  = torch.utils.data.DataLoader(ds_test,  batch_size=batch_size, shuffle=False)

In [ ]:
# データの中身を確認する．for文で中身を読み込む
for x,y in dl_train:
    break
print(y.shape)
#print(y)
print(x.shape)
#print(x[0])
print(y[0])
transforms.functional.to_pil_image(x[0].view(28, 28))  # 画像の確認

# Pytorchを用いたNNの学習
Pytorchを用いたNNの学習は以下の手順で実装できる．
1. 【モデル設計】nn.Moduleを用いてネットワークを設計する．
2. 【モデル準備】ネットワークをインスタンス化する．
3. 【訓練の準備】損失関数をインスタンス化する．
4. 【訓練の準備】最適化手法をインスタンス化する．
5. 【訓練】以下の手順をイテレーションする．
  1. 訓練データの一部$(x, y)$をサンプリングする．
  2. ネットワークに$x$を入力し予測結果$p$を得る．
  3. 損失関数を用いて$y$と$p$の誤差を算出する．
  4. 誤差を逆伝播させ，最適化手法を用いてネットワークの重みを更新する．
6. 【予測/評価】訓練済みモデルをテストデータで評価する．

## 【モデル設計】nn.Moduleを用いてネットワークを設計
- Pytorchでモデルを構築する際に一つのブロックをnn.Moduleのサブクラスとして定義して使用すると便利である．
- nn.Moduleはコンストラクタでレイヤをインスタンス化し，forward()で順伝播を定義する．

### nn.Moduleの定義
- nn.Moduleを継承したサブクラスを定義する．
- 基本的にはコンストラクタで必要な部品を調達し，forward()で処理を書く．
- 今回はシンプルな4層のMulti-layer Perceptron（MLP）を実装する．
 - 入力（1x28x28次元）→Reshape（748次元）→全結合層（512次元）→全結合層（256次元）→全結合層（128次元）→出力層（10次元）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 非常にシンプルな4層のMLPを設計する．
class SimpleNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNet, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        # nn.Sequential()に積層していく
        self.main = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        # (?, 1, 28, 28)で入力されるので(?, 784)に変形する28*28　メソッド
        x = x.view(-1, self.input_dim)
        # nn.Sequential()は層の塊（ブロック）として動作する
        return self.main(x)


In [ ]:
# SimpleNetは(748→512→256→128→10)と変形させるネットワークである．
# インスタンス化　実体化
net = SimpleNet(28*28, 10)  # 入力(748)→出力(10)
net

## 【モデル準備】ネットワークをインスタンス化
### nn.Moduleの利用
- nn.ModuleのサブクラスSimpleNetをインスタンス化して利用してみる．

In [ ]:
def get_default_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

# GPUを使用する。
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = get_default_device()

In [ ]:
# SimpleNetは(748→512→256→128→10)と変形させるネットワークである．
net = SimpleNet(28*28, 10)  # 入力(748)→出力(10)

# 出力を試しに確認してみる．
for x,y in dl_train: # 1イテレーション分のデータを取得
    break
out = net(x)
print("Xの書式", x.shape)
print("Shapeの確認", out.shape)
print("1データ目の出力", out[0])
transforms.functional.to_pil_image(x[0].view(28, 28))  # 画像の確認

- 上述のように，インスタンス化したnetは関数のように利用可能である．
```python
net = SimpleNet(28*28, 10)  # インスタンス化
out = net(x)                # 関数のように利用可能
```
- したがって，入力$x \in \mathbb{R}^{128 \times 1 \times 28 \times 28}$をnetに代入すると，
 - $\mathbb{R}^{128 \times 748}$の書式に変換される．
 - net.mainにより$\mathbb{R}^{128 \times 10}$の書式に変換される．
- 予測結果[0]と入力画像[0]の例も示している．
 - ただしnetは未訓練のため乱数が10個出力されるだけである．

## 【訓練の準備】損失関数と最適化手法をインスタンス化
### 訓練の準備
- Pytorchのモデルの訓練に必要な情報を事前準備する．
- 必要なのは以下の2点
 - 損失関数：今回は一般的な分類問題で用いられるCategorical Cross-Entropy Loss
 - 最適化手法（optimizer）：今回は標準的なOptimizerとしてAdamを学習率0.001で

In [ ]:
# 損失関数：Pytorch標準のbinary cross entropy lossを採用する．
criterion = nn.CrossEntropyLoss()

# 最適化手法：Adamをlr=0.001で利用する．
lr = 0.001
opt = torch.optim.Adam(net.parameters(), lr=lr) # netのパラメータを入れる．

## 【訓練】以下の手順をイテレーション
1. 訓練データの一部$(x, y)$をサンプリングする．
2. ネットワークに$x$を入力し予測結果$p$を得る．
3. 損失関数を用いて$y$と$p$の誤差を算出する．
4. 誤差を逆伝播させ，最適化手法を用いてネットワークの重みを更新する．

今回は1イテレーション（1 epoch）を関数化して実装する．

In [ ]:
# 1エポック実行
def train(net, opt, criterion, loader, device="cpu"):
    net.train() #NETを訓練モードにする
    running_loss = 0.0
    correct = 0
    # loader内のデータを1周訓練させる
    for i, (inputs, labels) in enumerate(loader, 0):
        opt.zero_grad()  # 勾配をゼロにする．
        inputs, labels = inputs.to(device), labels.to(device)
        
        out = net(inputs)   # Feed forward
        loss = criterion(out, labels)  # Calculate Loss
        loss.backward()     # Backward
        opt.step()    # Update weights
        
        correct += sum(out.argmax(dim=1)==labels)
        running_loss += loss.item()
    train_acc = (correct/len(loader.dataset)).cpu().item()
    return running_loss, train_acc

## 【予測/評価】訓練済みモデルをテストデータで評価
- 訓練中にテスト性能を評価したいので，評価関数を事前に作成しておく．
- 訓練データとは別のデータセット（テストデータ）を用いて訓練済みモデルの性能を評価する．
- 今回は評価を関数化して実装する．

In [ ]:
# 評価
def evaluate(net, loader, device="cpu"):
    net.eval()  # netを評価モードにする．

    with torch.no_grad():  # 評価なので勾配計算を不要とする．
        Xs, ys, preds = [], [], []
        for inputs, labels in loader:
            Xs.append(inputs)
            ys.append(labels.detach().numpy())
            preds.append(net(inputs.to(device)).to("cpu").detach().numpy())

    import numpy as np
    Xs = torch.cat(Xs, dim=0)
    ys = np.concatenate(ys, axis=0)
    preds = np.concatenate(preds, axis=0).argmax(axis=1)
    acc = (ys==preds).mean()
    return acc, (Xs, ys, preds)

### モデル全体の訓練
- 訓練はデータを1周するだけではなく，繰り返し学習することで行われる．
- 1周を1epochと呼ぶ．
- net.train()はPytorchには訓練時と検証時でモデルの挙動を変更するためのメソッドがある．

In [ ]:
from tqdm.notebook import trange
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

hist = []
n_epochs = 20  # 20 epochs訓練する

net = net.to(device)  # GPUで訓練できるようにする．
net.train()  # netを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(net, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(net, dl_test, device)
    hist.append([loss, train_acc, test_acc])

### 訓練結果の可視化
- Categorical Cross-entropy Lossを最小化するよう訓練が進んでいる．
- Epochごとの訓練の推移を可視化してみる．

In [ ]:
def plot_history(history):
    fig, ax1 = plt.subplots()

    # 損失の推移を描画する。
    ax1.plot(history[:, 0], label="train loss", linestyle="dashed", marker = 'o')
    ax1.set_xlabel("Epoch")
    
    ax2 = ax1.twinx()
    ax2.plot(history[:, 1], label="train acc", marker = 'o')
    ax2.plot(history[:, 2], label="test acc", marker = 'o')

    fig.legend(loc="center")
    plt.show()

plot_history(np.array(hist))
print(hist[-1])

- train lossとtrain accに着目すると，いずれも改善傾向が見られる（lossは減少，accは上昇）．
- test acc二着目すると，同様に改善傾向はあるがtrain_accに追いつけない傾向がある．
- もう少し訓練を進めること収束しそうである．

### 画像と予測結果の可視化
- せっかくなので画像と予測結果を可視化してみる．

In [ ]:
acc, (Xs, ys, preds) = evaluate(net.to(device), dl_test, device)

a, b =300, 350

# 画像を格子状に並べる。
img = torchvision.utils.make_grid(torch.Tensor(Xs[a:b]), nrow=10, normalize=True, pad_value=1)
# テンソルを PIL Image に変換する。
img = transforms.functional.to_pil_image(img)

print(ys[a:b].reshape(-1, 10))
print(preds[a:b].reshape(-1, 10))
print((ys[a:b]==preds[a:b]).reshape(-1,10))
img

## せっかくなので，CNNも試してみる．
- シンプルな4層のCNNを構築してみよう．
- カーネルサイズは大きめに5x5としておこう．
- そのままだと画像が小さくなり続けるのでpadding=2としてサイズ低下を防止する．
 - 入力（1x28x28）→Conv（32x28x28）→Pool（32x14x14）→Conv（64x14x14）→Pool（64x7x7）→Conv（128x7x7）→GAP→出力層（10次元） 
 - GAPはGlobal Average Poolingで，チャネルごとにAverage Poolingを行う．
   - これにより，入力画像サイズに依存しないモデルが実装可能となる．


- nn.Conv2DはAPIリファレンス https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html によると
``` python
Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, 
       padding_mode='zeros', device=None, dtype=None)
```
 - in_channels, out_channels, kernel_sizeだけ指定しておけば最低限問題なし．


- nn.AvgPool2dはAPIリファレンス https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html によると
```python
AvgPool2d(kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None)
```
 - kernel_size，strideを指定するのが一般的である．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 非常にシンプルな4層のCNN（Encoder3層＋出力層）を設計する．
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(SimpleCNN, self).__init__()
        self.in_channels = in_channels
        self.output_dim = output_dim
        # nn.Sequential()に積層していく
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, 5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(64, 128, 5, padding=2),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  #  サイズが（1x1）になるようにAvgPoolingする
        x = x.view(-1, 128)
        return self.classifier(x)


In [ ]:
print(device)
# 関数等は使い回すが，各インスタンスは再生成する必要がある．
cnn = SimpleCNN(1, 10)
criterion = nn.CrossEntropyLoss()
lr = 0.001
opt = torch.optim.Adam(cnn.parameters(), lr=lr) # cnnのパラメータを入れる．

hist_cnn = []
n_epochs = 20  # 20 epochs訓練する

cnn = cnn.to(device)  # GPUで訓練できるようにする．
cnn.train()  # netを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(cnn, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(cnn, dl_test, device)
    hist_cnn.append([loss, train_acc, test_acc])

print(hist_cnn[-1])
plot_history(np.array(hist_cnn))

In [ ]:
# 出力を試しに確認してみる．
for x,y in dl_test: # 1イテレーション分のデータを取得
    break
net = net.to("cpu")
out = net(x)
print("Xの書式", x.shape)
print("Shapeの確認", out.shape)
print("1データ目の出力", out[0])
transforms.functional.to_pil_image(x[0].view(28, 28))  # 画像の確認

## 同じ要領でVGGを作ってみよう．
- 上述したSimpleCNNをコピペし，VGGライクなネットワークを構築して訓練してみよう．
- 訓練部分や可視化部分もコピペで良い．
- 詳細なパラメータは原著論文を参照されたい．
  - https://arxiv.org/abs/1409.1556
- ただし，今回画像サイズが28x28の都合もあり，Pooingをstride=2で実施すると精々4回が限度である．
  - そこで3Block程度の小さなVGGを作ることにしよう．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 書くのが面倒になるのでnn.Moduleにまとめておく．
class CR(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CR, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1),  # 画像サイズを落とさないようにpadding
            nn.ReLU()
        )
    def forward(self, x):
        return self.block(x)

# VGG16をベースに，Pooling時のStrideを変更したVer
class smallVGG(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(smallVGG, self).__init__()
        self.in_channels = in_channels
        self.output_dim = output_dim
        # nn.Sequential()に積層していく
        self.encoder = nn.Sequential(
            CR(in_channels, 16),  # フィルタも減らしておく
            CR(16, 16),
            nn.MaxPool2d(2, 2),
            CR(16, 32),
            CR(32, 32),
            nn.MaxPool2d(2, 2),
            CR(32, 64),
            CR(64, 64),
            CR(64, 64),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  #  サイズが（1x1）になるようにAvgPoolingする
        x = x.view(-1, 64)
        return self.classifier(x)


In [ ]:
print(device)
# 関数等は使い回すが，各インスタンスは再生成する必要がある．
cnn = smallVGG(1, 10)
criterion = nn.CrossEntropyLoss()
lr = 0.001
opt = torch.optim.Adam(cnn.parameters(), lr=lr) # cnnのパラメータを入れる．

hist_cnn = []
n_epochs = 20  # 20 epochs訓練する

cnn = cnn.to(device)  # GPUで訓練できるようにする．
cnn.train()  # netを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(cnn, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(cnn, dl_test, device)
    hist_cnn.append([loss, train_acc, test_acc])

print(hist_cnn[-1])
plot_history(np.array(hist_cnn))

## ResNetライクなモデルも作ってみよう
- ResNetを通じて，BatchNormalizationとSkipConnectionという重要な概念の利用方法を学ぼう．
- 詳細なパラメータは原著論文を参照されたい．
  - https://arxiv.org/abs/1512.03385

### 練習問題
- 上のVGGライクモデルを改良してResNetライクに変更してみよう．
- 今回の改良ポイントはconvreluにBatchNormを挿入して，conv-bn-reluにすることと，
- 各Blockの前後にショートカットゲートを挿入することに挑戦しよう．
- 少し難しいと思うので，1度自分で考えてみる→ググって調べてみる→動画で解説を見ると進めていこう．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 書くのが面倒になるのでnn.Moduleにまとめておく．
class CBR(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CBR, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1),  # 画像サイズを落とさないようにpadding
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
    def forward(self, x):
        return self.block(x)

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, rep=2) -> None:
        super(BasicBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.rep = rep
        layers = []
        inch = in_channels
        for i in range(rep):
            layers.append(CBR(inch, out_channels))
            inch = out_channels
        self.block = nn.Sequential(*layers)
        self.pwconv = nn.Conv2d(in_channels, out_channels, 1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        out = self.block(x)
        out = self.pwconv(x) + out
        return self.pool(out)


# VGG16をベースに，Pooling時のStrideを変更したVer
class smallResNet(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(smallResNet, self).__init__()
        self.in_channels = in_channels
        self.output_dim = output_dim
        # nn.Sequential()に積層していく
        self.encoder = nn.Sequential(
           BasicBlock(in_channels, 16, rep = 2),
           BasicBlock(16, 32, rep = 2),
           BasicBlock(32, 64, rep = 3)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  #  サイズが（1x1）になるようにAvgPoolingする
        x = x.view(-1, 64)
        return self.classifier(x)

In [ ]:
from tqdm.notebook import trange

print(device)
# 関数等は使い回すが，各インスタンスは再生成する必要がある．
cnn = smallResNet(1, 10)
criterion = nn.CrossEntropyLoss()
lr = 0.001
opt = torch.optim.Adam(cnn.parameters(), lr=lr) # cnnのパラメータを入れる．

hist_cnn = []
n_epochs = 20  # 20 epochs訓練する

cnn = cnn.to(device)  # GPUで訓練できるようにする．
cnn.train()  # netを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(cnn, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(cnn, dl_test, device)
    hist_cnn.append([loss, train_acc, test_acc])

print(hist_cnn[-1])
plot_history(np.array(hist_cnn))

## SE Netライクなモデルを実装してみよう
- SE Netの実装を通じて，BatchNormalizationとSkipConnectionという重要な概念の利用方法を学ぼう．
- 詳細なパラメータは原著論文を参照されたい．
  - https://arxiv.org/abs/1709.01507

### 練習問題
- 上のResNetライクモデルを改良してSE-blockを挿入してみよう．
- 今回の改良ポイントはSE-Block(論文中のFig1)を各モジュールの末尾に挿入することである．
- 少し難しいと思うので，1度自分で考えてみる→ググって調べてみる→動画で解説を見ると進めよう．

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 書くのが面倒になるのでnn.Moduleにまとめておく．
class CBRSE(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CBRSE, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1),  # 画像サイズを落とさないようにpadding
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            SE(out_channels, r=8)
        )
    def forward(self, x):
        return self.block(x)

class SE(nn.Module):
    def __init__(self, in_channels, r = 8):
        super(SE, self).__init__()
        self.in_channels = in_channels
        self.r = r
        self.c1 = nn.Conv2d(in_channels, in_channels//r, 1)
        self.c2 = nn.Conv2d(in_channels//r, in_channels, 1)

    def forward(self, x):
        out = F.adaptive_avg_pool2d(x, (1, 1))  #  サイズが（1x1）になるようにAvgPoolingする
        out = F.relu(self.c1(out))
        out = F.sigmoid(self.c2(out))
        return x * out


class SEBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, rep=2) -> None:
        super(SEBasicBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.rep = rep
        layers = []
        inch = in_channels
        for i in range(rep):
            layers.append(CBRSE(inch, out_channels))
            inch = out_channels
        self.block = nn.Sequential(*layers)
        self.pwconv = nn.Conv2d(in_channels, out_channels, 1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        out = self.block(x)
        out = self.pwconv(x) + out
        return self.pool(out)


# VGG16をベースに，Pooling時のStrideを変更したVer
class smallSEResNet(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(smallSEResNet, self).__init__()
        self.in_channels = in_channels
        self.output_dim = output_dim
        # nn.Sequential()に積層していく
        self.encoder = nn.Sequential(
           SEBasicBlock(in_channels, 16, rep = 2),
           SEBasicBlock(16, 32, rep = 2),
           SEBasicBlock(32, 64, rep = 3)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  #  サイズが（1x1）になるようにAvgPoolingする
        x = x.view(-1, 64)
        return self.classifier(x)

In [ ]:
print(device)
# 関数等は使い回すが，各インスタンスは再生成する必要がある．
cnn = smallSEResNet(1, 10)
criterion = nn.CrossEntropyLoss()
lr = 0.001
opt = torch.optim.Adam(cnn.parameters(), lr=lr) # cnnのパラメータを入れる．

hist_cnn = []
n_epochs = 20  # 20 epochs訓練する

cnn = cnn.to(device)  # GPUで訓練できるようにする．
cnn.train()  # netを訓練モードにする．

for epoch in trange(n_epochs, desc="epoch"):  # 訓練
    loss, train_acc = train(cnn, opt, criterion, dl_train, device)
    test_acc, _ = evaluate(cnn, dl_test, device)
    hist_cnn.append([loss, train_acc, test_acc])

print(hist_cnn[-1])
plot_history(np.array(hist_cnn))